In [41]:
%load_ext autoreload
%autoreload 2
import sys
import pandas as pd
import pickle
import itertools 
sys.path.append("../")
sys.path.append("../title_maker_pro")
from title_maker_pro import datasets
from collections import OrderedDict
import torch
from transformers import AutoModelWithLMHead, AutoTokenizer
import copy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
dataset_path = "/mnt/evo/projects/title-maker-pro/data/urban_dictionary_words.pickle"
with open(dataset_path, 'rb') as f:
    dataset = pickle.load(f)

In [80]:
blacklist = datasets.Blacklist.load("/mnt/evo/projects/title-maker-pro/models/blacklist.pickle")

In [21]:
pd_dataset = pd.DataFrame(
    (
        (d.word, d.meaning, d.examples[0], d.upvotes, d.downvotes, d.creation_epoch) 
        for d in itertools.chain.from_iterable(e.definitions for e in dataset.values())
    ),
    columns=["word", "meaning", "example", "upvotes", "downvotes", "creation_epoch"]
)

In [99]:
def cut(original, f, name):
    n = original[f]
    print(f"{name} cut by {100 * len(n) / (len(original)):.2f}% ({len(original)} -> {len(n)})")
    return n

In [103]:
t = pd_dataset.copy()
t = cut(t, ~(pd_dataset["word"].apply(blacklist.contains)), name="blacklist")
t = cut(t, ((pd_dataset["example"].str.len() + pd_dataset["meaning"].str.len() + pd_dataset["word"].str.len()) < 250), name="length")
t = cut(t, (pd_dataset["upvotes"] >= 4), name="upvotes")

blacklist cut by 49.63% (2961824 -> 1469932)


/home/tdimson/anaconda3/envs/company_makeup/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


length cut by 65.20% (1469932 -> 958444)
upvotes cut by 47.83% (958444 -> 458387)


In [148]:
cleaned_dataset = OrderedDict()
i = 0
num_defns = 0
for k, ud_word in dataset.items():
    good_defns = []
    for d in ud_word.definitions:
        if i in valid_indexes:
            good_defns.append(d)
            num_defns += 1
        i += 1
    
    if good_defns:
        new = copy.deepcopy(ud_word)
        new.definitions = good_defns
        cleaned_dataset[k] = new

In [150]:
cleaned_dataset_path = "/mnt/evo/projects/title-maker-pro/data/urban_dictionary_250_cleaned.pickle"
with open(cleaned_dataset_path, "wb") as f:
    pickle.dump(cleaned_dataset, f, pickle.HIGHEST_PROTOCOL)

In [52]:
# nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos', use)
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens(datasets.SpecialTokens.special_tokens_dict())
blacklist = datasets.Blacklist.load("/mnt/evo/projects/title-maker-pro/models/blacklist_urban_dictionary.pickle")
model = AutoModelWithLMHead.from_pretrained("/mnt/evo/projects/title-maker-pro/models/urban_dictionary_250_cleaned_lr_00005_b9_seed4/checkpoint-60000").to("cpu")

In [55]:
words, stats = datasets.UrbanDictionaryDataset.generate_words(
    tokenizer, model,
    num=5,
    max_iterations=5, 
    blacklist=blacklist, 
    generation_args=dict(
        top_k=200,
        num_return_sequences=5,
        max_length=250,
        do_sample=True,
    ),
    dedupe_titles=True,
    filter_proper_nouns=True,
    min_definition_words=3,
)

In [56]:
print(stats)
print()
datasets.GeneratedWord.print_words(words)

iterations=4 time=13.484499454498291 stanza_time=0.0 | items_considered 1.00@19, failed_match 0.53@10, blacklist_filtered 0.00@0, seen_filtered 0.00@0, proper_noun_filtered 0.11@2, example_missing 0.00@0, short_definitions 0.00@0, example_missing_title 0.11@2, example_pos_match_failed 0.00@0, user_filtered 0.00@0, returned 0.26@5

sogdakapoo
	Basically a guy, pretty much man like. So basically... a girl, who, when she gets at the end of their contact line, says something
	"Oh, my god, look! It's Sogdakapoo!
	 
	He used to be such a skank, 'cause he sucked all my dick and he called it his.
	 
	In the bad part on the break! I caught him snoring with his sack!"
----------------
awesomsqueeblee
4. The plural of awesomenessid very easily."d, or somthing like that.
	"awesomsqueeblee-nawesomemephyteezomemephedamfukkie 
	when someone says something that is abresolutly awesome."
----------------
gifit
 Also can be used to describe the object in question. also can be used as an adjective.
	"1. w